In [16]:
import os
import zipfile
import pandas as pd
import cv2

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
%cd 'gdrive/' 
#cd command is used to change the directory 

[Errno 2] No such file or directory: 'gdrive/'
/content/gdrive


In [11]:
#If the following path does not exist create it.
path = '/content/gdrive/MyDrive/Traffic_Sign_Detection_YOLOv4' # Set(folder) Traffic_Sign_Detection inside Google Drive
isdir = os.path.isdir(path) 
if not isdir: #If directory is not present 
  os.mkdir('/content/gdrive/MyDrive/Traffic_Sign_Detection_YOLOv4') # make the directory (folder)
else:
  print("Folder already exist")

already exist


In [15]:
if os.getcwd() == '/content':
  %cd '/content/gdrive/'
if os.getcwd() == '/content/gdrive':
  %cd '/content/gdrive/MyDrive/' 
if os.getcwd() == '/content/gdrive/MyDrive':
  %cd 'Traffic_Sign_Detection_YOLOv4/' 
print("Current working directory is now:")
print(os.getcwd())

/content/gdrive/MyDrive
/content/gdrive/MyDrive/Traffic_Sign_Detection_YOLOv4
Current working directory is now:
/content/gdrive/MyDrive/Traffic_Sign_Detection_YOLOv4


In [ ]:
#As the ts.zip folder size is big so it cannot be uploaded to github.
#It is already present in the Dataset downloaded. Upload the zip file (ts.zip) to google drive inside Traffic_Sign_Detection_YOLOv4 folder
#Now unzip the folder. Uncomment below lines to run the code. Only do it once if ts folder already extracted do not do it.
unzip = zipfile.ZipFile('ts.zip')
unzip.extractall('ts')

In [ ]:
full_path_to_ts_dataset = '/content/gdrive/MyDrive/Traffic_Sign_Detection_YOLOv4/ts' #set path to ts folder

In [ ]:
# Defining lists for categories according to the classes ID's
# Prohibitory category:
# circular Traffic Signs with white background and red border line
p = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16] #Here p stands for prohibitory

# Danger category:
# triangular Traffic Signs with white background and red border line
d = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

# Mandatory category:
# circular Traffic Signs with blue background
m = [33, 34, 35, 36, 37, 38, 39, 40]

# Other category:
o = [6, 12, 13, 14, 17, 32, 41, 42]

In [ ]:
#Loading original annotations into Pandas dataFrame
# Reading txt file with annotations separated by semicolons
# If you're using Windows system offline Jupyter Notebook and not Google Colab, it might need to change
# this: + '/' +
# to this: + '\' +
# or to this: + '\\' +
ann = pd.read_csv(full_path_to_ts_dataset + '/' + 'gt.txt',
                  names=['ImageID',
                         'XMin',
                         'YMin',
                         'XMax',
                         'YMax',
                         'ClassID'],
                  sep=';')

In [ ]:
#Calculating numbers for YOLO format without normalization
#Adding new empty columns to dataFrame to save numbers for YOLO format
ann['CategoryID'] = ''
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

In [ ]:
# Getting category's ID according to the class's ID
# Writing numbers into appropriate column
ann.loc[ann['ClassID'].isin(p), 'CategoryID'] = 0
ann.loc[ann['ClassID'].isin(d), 'CategoryID'] = 1
ann.loc[ann['ClassID'].isin(m), 'CategoryID'] = 2
ann.loc[ann['ClassID'].isin(o), 'CategoryID'] = 3


In [ ]:
#Calculating bounding box's center in x and y for all rows
# Saving results to appropriate columns
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

# Calculating bounding box's width and height for all rows
# Saving results to appropriate columns
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

In [ ]:
#Getting Pandas dataFrame that has only needed columns
# By using 'loc' method we locate here all rows
# but only specified columns
# By using copy() we create separate dataFrame
# not just a reference to the previous one
# and, in this way, initial dataFrame will not be changed
r = ann.loc[:, ['ImageID',
                'CategoryID',
                'center x',
                'center y',
                'width',
                'height']].copy()

In [ ]:
# Normalizing YOLO numbers according to the real image width and height
# Saving annotations in txt files
# Converting images from ppm to jpg

In [ ]:
os.chdir(full_path_to_ts_dataset) # Changing the current directory
# to ts folder that contain images

In [ ]:
# Checking if the current directory is now ts folder
# print(os.getcwd())

In [ ]:
# Using os.walk for going through all directories
# and files in them from the current directory
# Fullstop in os.walk('.') means the current directory
for current_dir, dirs, files in os.walk('.'):
    # Going through all files
    for f in files:
        # Checking if filename ends with '.ppm'
        if f.endswith('.ppm'):
            # Reading image and getting its real width and height
            image_ppm = cv2.imread(f)

            # Slicing from tuple only first two elements
            h, w = image_ppm.shape[:2]

            # Slicing only name of the file without extension
            image_name = f[:-4]

            # Getting Pandas dataFrame that has only needed rows
            # By using 'loc' method we locate needed rows
            # that satisfies condition 'classes['ImageID'] == f'
            # that is 'find from the 1st column element that is equal to f'
            # By using copy() we create separate dataFrame
            # not just a reference to the previous one
            # and, in this way, initial dataFrame will not be changed
            sub_r = r.loc[r['ImageID'] == f].copy()

            # Normalizing calculated bounding boxes' coordinates
            # according to the real image width and height
            sub_r['center x'] = sub_r['center x'] / w
            sub_r['center y'] = sub_r['center y'] / h
            sub_r['width'] = sub_r['width'] / w
            sub_r['height'] = sub_r['height'] / h

            # Getting resulted Pandas dataFrame that has only needed columns
            # By using 'loc' method we locate here all rows
            # but only specified columns
            # By using copy() we create separate dataFrame
            # not just a reference to the previous one
            # and, in this way, initial dataFrame will not be changed
            resulted_frame = sub_r.loc[:, ['CategoryID',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

            # Checking if there is no any annotations for current image
            if resulted_frame.isnull().values.all():
                # Skipping this image
                continue

            # Preparing path where to save txt file
            # Pay attention! If you're using Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            path_to_save = full_path_to_ts_dataset + '/' + image_name + '.txt'

            # Saving resulted Pandas dataFrame into txt file
            resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

            # Preparing path where to save jpg image
            # Pay attention! If you're using Windows, it might need to change
            # this: + '/' +
            # to this: + '\' +
            # or to this: + '\\' +
            path_to_save = full_path_to_ts_dataset + '/' + image_name + '.jpg'

            # Saving image in jpg format by OpenCV function
            # that uses extension to choose format to save with
            cv2.imwrite(path_to_save, image_ppm)